In [591]:
%pip install groq
%pip install python-pptx

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [592]:
query="effect of demonitatization in india"

In [593]:
from groq import Groq

client = Groq(api_key="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")
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": "generate / rectify grammar - give a one line response - neither too long nor too short (maximum 5 words) - just the title"
        },
        {
            "role": "user",
            "content": f"query[{query}]"
        }
    ],
    temperature=1,
    max_tokens=8192,
    top_p=1,
    stream=True,
    stop=None,
)
query=""
for chunk in completion:
    query+=chunk.choices[0].delta.content or ""
print(query)

Impact of Demonetization on India


In [594]:
slides=5

In [595]:
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": f"generate {slides} ppt slide titles in order (keep them sweet and short) based on the query[] - split the titles with delimiter '%%%' and return all titles within a codeblock ``` - do not mention slide count - do not include asterisks * - dont generate question tags - max words - (6 to 8) - dont include double quotes ,just plain text"
        },
        {
            "role": "user",
            "content": f"query[{query}]"
        }
    ],
    temperature=1,
    max_tokens=8192,
    top_p=1,
    stream=True,
    stop=None,
)
slide_points_full=""
for chunk in completion:
    slide_points_full+=chunk.choices[0].delta.content or ""


In [596]:
print(slide_points_full)

```
Impact of Demonetization on India %%% 
Rise of Digital Payments in India %%%
Economic Impact of Demonetization %%%
Impact of Demonetization on MSMEs %%% 
Long-term Effects of Demonetization
```


In [597]:
slide_points=slide_points_full[slide_points_full.find('```')+4:slide_points_full.rfind('```')-1].replace('\n',"").split("%%%")

In [598]:
print(slide_points)

['Impact of Demonetization on India ', ' Rise of Digital Payments in India ', 'Economic Impact of Demonetization ', 'Impact of Demonetization on MSMEs ', ' Long-term Effects of Demonetization']


In [599]:
slide_content=[]

In [600]:
for point in slide_points:
    if point.strip()!="":
        completion = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {
                    "role": "system",
                    "content": f"generate a paragraph (50-60 words STRICTLY) for slide content ONLY (neither too big nor too small) for title[] - return the body only without any unwanted text **do not use linebreak** - do not use asterisks* - use justify text align - do not mention anything else, give only the slide content paragraph as a response - return only information related to the point[] not statements like 'here is..','some of ...' etc - do not use linebreak generate entire response in one line - dont use asterisks * "
                },
                {
                    "role": "user",
                    "content": f"[{point}]"
                }
            ],
            temperature=1,
            max_tokens=8192,
            top_p=1,
            stream=True,
            stop=None,
        )
        temp_slide_content=""
        for chunk in completion:
            temp_slide_content+=chunk.choices[0].delta.content or ""
        slide_content.append(temp_slide_content)

In [601]:
print(slide_content)

["The government's surprise announcement of demonetizing high-denomination banknotes led to a massive disruption in the country's economy, resulting in unprecedented queues at ATMs, a shortage of cash, and significant economic costs, with estimates suggesting a 0.5% reduction in GDP growth rate and nearly 1 million job losses in the informal sector.", 'Rise of digital payments in India, driven by UPI, has revolutionized the way Indians transact, with over 2 billion transactions per month, facilitating financial inclusion for millions, boosting e-commerce growth, and fostering a digitally savvy population, making India a hub for fintech innovation and transformation.', 'The economic impact of demonetization affected various sectors, including a significant decline in consumer spending, reduced industrial production, and a sharp increase in unemployment, leading to a substantial decrease in GDP, while the cost of printing new currency notes and absorbing excess liquidity contributed to r

In [602]:
from pptx import Presentation
from pptx.util import Inches
from PIL import Image
from pptx.dml.color import RGBColor

slide_width = Inches(10)  
slide_height = Inches(5.625) 

prs = Presentation()

prs.slide_width = slide_width
prs.slide_height = slide_height

slide = prs.slides.add_slide(prs.slide_layouts[5])  

img_path = 'styles/template1/1.png'
img = Image.open(img_path)
img_width, img_height = img.size

max_width = slide_width  
max_height = slide_height 
scale_width = max_width / img_width
scale_height = max_height / img_height
scale = min(scale_width, scale_height)

left = (slide_width - img_width * scale) / 2
top = (slide_height - img_height * scale) / 2

pic = slide.shapes.add_picture(img_path, left, top, width=img_width * scale, height=img_height * scale)
title_width = Inches(8) 
title_left = (slide_width - title_width) / 2
title = slide.shapes.add_textbox(title_left, Inches(2), title_width, Inches(1))
title.text = query
title.text_frame.word_wrap = True
title.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255) 
title.text_frame.paragraphs[0].font.bold = True  
title.text_frame.paragraphs[0].font.size = Inches(0.4)  
title.text_frame.paragraphs[0].font.name = 'Courier New' 
title.text_frame.paragraphs[0].alignment = 2

In [603]:
img_path2 = 'styles/template1/2.png'
for count in range(len(slide_points)):
    slide = prs.slides.add_slide(prs.slide_layouts[5])  

    left = (slide_width - img_width * scale) / 2
    top = (slide_height - img_height * scale) / 2

    pic = slide.shapes.add_picture(img_path2, left, top, width=img_width * scale, height=img_height * scale)

    title = slide.shapes.add_textbox(left, Inches(0.2), img_width * scale, Inches(1))
    title.text = slide_points[count]
    title.text_frame.word_wrap = True
    title.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255) 
    title.text_frame.paragraphs[0].font.bold = True
    title.text_frame.paragraphs[0].font.size = Inches(0.3) 
    title.text_frame.paragraphs[0].font.name = 'Courier New' 
    title.text_frame.paragraphs[0].alignment = 2
    title = slide.shapes.add_textbox(left+Inches(0.25), Inches(1.5), Inches(4), Inches(1))
    title.text = slide_content[count]
    title.text_frame.word_wrap = True  
    title.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255) 
    title.text_frame.paragraphs[0].font.bold = False
    title.text_frame.paragraphs[0].font.size = Inches(0.15) 
    title.text_frame.paragraphs[0].font.name = 'Courier New' 
    title.text_frame.paragraphs[0].alignment = 2

prs.save('test.pptx')

In [604]:
%pip install -qU langchain-text-splitters
%pip install pypdf
%pip install sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [605]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    separators=["\n\n", "\n", " ", ""],
)
from pypdf import PdfReader

reader = PdfReader("test.pdf")
pages=len(reader.pages)
document_data=""
for i in range(pages):
    document_data+=" "+reader.pages[i].extract_text()
texts = text_splitter.create_documents([document_data])
texts2=[]
for i in texts:
    texts2.append(i.page_content)


In [606]:
print(texts2)

['Company Strategy Overview:', "In today's meeting, we'll delve into our company's strategic direction to ensure continued growth", 'continued growth and', 'success. Our primary focus remains on innovation, customer satisfaction, and operational', 'and operational excellence.', 'Innovation:', 'We recognize the importance of staying ahead in a competitive market. Our commitment to innovation', 'drives us to constantly explore new technologies, products, and services. By fostering a culture of', 'a culture of', 'creativity and experimentation, we aim to develop groundbreaking solutions that meet evolving', 'that meet evolving customer', 'needs. \n \nCustomer Satisfaction:', 'Our customers are at the heart of everything we do. We prioritize delivering exceptional', 'exceptional experiences and', 'exceeding expectations. Through proactive engagement and personalized solutions, we aim to build', 'long-lasting relationships and drive loyalty. \n \nOperational Excellence:', 'Efficiency and ef

In [607]:
%pip install langchain_community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [608]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
doc_result = embeddings.embed_documents(texts2)

/home/sid/Desktop/hub/PPT-AI/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [609]:
vectors = []

for i in range(len(doc_result)):
    vectors.append(
        {
            "id": f"vec{i+1}", 
            "values": doc_result[i], 
            "metadata": {'text': texts2[i]}
        }
    )


In [611]:
%pip install pinecone_client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [612]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="d1eccc8c-3147-4d5e-992e-81b0c5d73278")
pc.create_index(
    name="quickstart",
    dimension=384, 
    metric="euclidean",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2024-04', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '9ce6b3e6411ac29f337da772c30b8f03', 'Date': 'Sun, 12 May 2024 15:50:45 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [613]:
index = pc.Index("quickstart")
index.upsert(
    vectors=vectors,
    namespace= "ns1"

)

{'upserted_count': 30}

In [614]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [615]:
query = "what about satisfying the customers with engagement"

In [617]:
xq = model.encode(query).tolist()
xc=index.query(
    namespace="ns1",
    vector=xq,
    top_k=10,
    include_values=True,
    include_metadata=True
)
relevant_chunk=""
for result in xc['matches']:
    relevant_chunk+=" "+result['metadata']['text']
print(relevant_chunk)

 exceeding expectations. Through proactive engagement and personalized solutions, we aim to build needs. 
 
Customer Satisfaction: Our customers are at the heart of everything we do. We prioritize delivering exceptional that meet evolving customer success. Our primary focus remains on innovation, customer satisfaction, and operational Expand market reach through targeted marketing campaigns and strategic partnerships. long-lasting relationships and drive loyalty. 
 
Operational Excellence: Enhance product offerings with a focus on user-centric design and functionality. and capitalize on opportunities. Together, we will drive our company forward and achieve shared We recognize the importance of staying ahead in a competitive market. Our commitment to innovation


In [618]:
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": f"answer the query[] based on the relevant_chunk[]"
        },
        {
            "role": "user",
            "content": f"query[{query}] relevant_chunk[{relevant_chunk}]"
        }
    ],
    temperature=0.1,
    max_tokens=8192,
    top_p=1,
    stream=True,
    stop=None,
)
relevant_response=""
for chunk in completion:
    relevant_response+=chunk.choices[0].delta.content or ""
print(relevant_response)

Based on the "relevant_chunk", it seems that the query "what about satisfying the customers with engagement" is referring to the following information:

"We aim to build long-lasting relationships and drive loyalty through proactive engagement and personalized solutions."

This suggests that the company prioritizes satisfying customers by being proactive and engaging with them in a personalized manner, which in turn builds trust and loyalty.
